In [ ]:
import numpy as np 
import pandas as pd 
import os
import cv2
from PIL import Image
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import tensorflow as tf 
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                         GlobalAveragePooling2D, SeparableConv2D,\
ZeroPadding2D, Convolution2D, ZeroPadding2D, Conv2DTranspose,ReLU, UpSampling2D, Concatenate, Conv2DTranspose, GlobalMaxPooling2D
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Conv2D, Activation, Multiply, Add, Input, BatchNormalization, GlobalAveragePooling2D, Dense

# Storing in a dataframe

In [ ]:

data= {}
root='/.../your_dataset_folder'
for i in os.listdir(root):
    for j in os.walk(root+i):
        for k in j[2]:
            data[root+i+'/'+k]=i


data=pd.DataFrame(data.items(),columns=['file','type'])
data=data.sample(frac=1)
data['type'].value_counts()

# Data splitting

In [ ]:
def split_data(data,ratio):
    last=int(len(data)*ratio)
    return data[:last], data[last:]

train,test=split_data(data,.8)

In [ ]:
size=224
batch_size=4

# Generators

In [ ]:
train_datagen = ImageDataGenerator(
rescale=1./255
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    
)

train_images = train_datagen.flow_from_dataframe(
    dataframe=train,
    x_col='file',
    y_col='type',
    target_size=(size, size),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42,
)

test_images = test_datagen.flow_from_dataframe(
    dataframe=test,
    x_col='file',
    y_col='type',
    target_size=(size, size),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,
    seed=42,
)

# Model construction

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Concatenate



def channel_attention_module(x, ratio=8):
    batch, _, _, channel = x.shape

    
    l1 = Dense(channel//ratio, activation="relu", use_bias=False)
    l2 = Dense(channel, use_bias=False)

    
    x1 = GlobalAveragePooling2D()(x)
    x1 = l1(x1)
    x1 = l2(x1)

    
    x2 = GlobalMaxPooling2D()(x)
    x2 = l1(x2)
    x2 = l2(x2)

    
    feats = x1 + x2
    feats = Activation("sigmoid")(feats)
    feats = Multiply()([x, feats])

    return feats

def spatial_attention_module(x):
    
    x1 = tf.reduce_mean(x, axis=-1)
    x1 = tf.expand_dims(x1, axis=-1)

    
    x2 = tf.reduce_max(x, axis=-1)
    x2 = tf.expand_dims(x2, axis=-1)

    
    feats = Concatenate()([x1, x2])
    
    feats = Conv2D(1, kernel_size=7, padding="same", activation="sigmoid")(feats)
    feats = Multiply()([x, feats])

    return feats

def cbam_block(x):
    x = channel_attention_module(x)
    x = spatial_attention_module(x)
    return x



def aspp_block(input_layer, dilation_rates=[1, 5, 7, 9], num_filters=128):
    aspp_layers = []
    for dilation_rate in dilation_rates:
        aspp_layer = Conv2D(num_filters, (3, 3), padding='same', activation='relu', dilation_rate=dilation_rate)(input_layer)
        aspp_layers.append(aspp_layer)
    concatenated = Concatenate()(aspp_layers)

    return concatenated

def create_aspp_model(input_shape):
    input_layer = Input(shape=input_shape)

    
    aspp_block1 = aspp_block(input_layer)
    aspp_block1=cbam_block(aspp_block1)
    maxpool1 = MaxPooling2D(pool_size=(2, 2))(aspp_block1)
    maxpool1=tf.keras.layers.BatchNormalization()(maxpool1)
    maxpool1= Dropout(.2)(maxpool1)
    
    aspp_block2 = aspp_block(maxpool1,num_filters=256)
    aspp_block2=cbam_block(aspp_block2)
    maxpool2 = MaxPooling2D(pool_size=(2, 2))(aspp_block2)
    maxpool2=tf.keras.layers.BatchNormalization()(maxpool2)
    maxpool2= Dropout(.1)(maxpool2)
    
    
    aspp_block3 = aspp_block(maxpool2,num_filters=256)
    aspp_block3=cbam_block(aspp_block3)
    maxpool3 = MaxPooling2D(pool_size=(2, 2))(aspp_block3)
    maxpool3=tf.keras.layers.BatchNormalization()(maxpool3)
    maxpool3= Dropout(.1)(maxpool3)
    
    aspp_block4 = aspp_block(maxpool3,num_filters=256)
    aspp_block4=cbam_block(aspp_block4)
    maxpool4 = MaxPooling2D(pool_size=(2, 2))(aspp_block4)
    maxpool4=tf.keras.layers.BatchNormalization()(maxpool4)
    maxpool4= Dropout(.1)(maxpool4)
    
    aspp_block5 = aspp_block(maxpool4,num_filters=256)
    aspp_block5=cbam_block(aspp_block5)
    
    
    
    aspp_model = tf.keras.Model(inputs=input_layer, outputs=aspp_block5)

    return aspp_model

input_shape = (224, 224, 3)  

model = create_aspp_model(input_shape)

x=model.output
x=tf.keras.layers.BatchNormalization()(x)
x= GlobalAveragePooling2D()(x)
model= tf.keras.Model(model.input,x)


# model.summary()


# ViT

In [ ]:
!pip install --quiet vit-keras
from vit_keras import vit

vit_base = vit.vit_b32(
        image_size = size,
        activation = 'softmax',
        pretrained = True,
        include_top = False,
        pretrained_top = False,
        )
for i in vit_base.layers:
    i.trainable=False
    
x =vit_base.output
x=tf.keras.layers.BatchNormalization()(x)
x = Dense(512, activation='selu')(x)
x= Dropout(.3)(x)
x=Dense(256,activation='selu')(x)
x= Dropout(.3)(x)
x_vit_base = Dense(128, activation='selu')(x)


x=model.output
x= Dropout(.3)(x)
x= Dense(256, activation='selu')(x)
x= Dropout(.3)(x)
x_cnn = Dense(128, activation='selu')(x)
x = Concatenate()([x_cnn, x_vit_base])


x= Dense(128, activation='selu')(x)
x= Dropout(.3)(x)
x= Dense(64, activation='selu')(x)
x= Dropout(.3)(x)
x= Dense(32, activation='selu')(x)
x= Dense(len(train_images.class_indices), activation='softmax')(x)

model= tf.keras.Model([model.input, vit_base.input],x)


# Compiling and Training

In [ ]:
def generator_two_img(gen):
    while True:
        X1i = gen.next()
        yield [X1i[0],X1i[0]], X1i[1]

In [ ]:
model.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.experimental.Adamax(),metrics=['accuracy'])
history = model.fit(
    generator_two_img(train_images),
    validation_batch_size=batch_size,
    steps_per_epoch=train_images.n//batch_size,
    batch_size=batch_size,
    epochs=100,
)

In [ ]:
model.save('model.h5')

In [ ]:
class_=train_images.class_indices.keys()
def prediction_generator(test_images):
    k=0
    for x in test_images:
        k+=len(x[1])
        yield [x[0],x[0]], x[1]
        if(k>=test_images.n):
            break

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,matthews_corrcoef
proba=model.predict(prediction_generator(test_images))
predicted = np.argmax(proba,axis=1)
true=test_images.labels
print("acc: ", accuracy_score(predicted, true))
print("preci: ", precision_score(predicted, true,average='macro'))
print("recall: ", recall_score(predicted, true,average='macro'))
print("f1: ", f1_score(predicted, true,average='macro'))
print("mcc: ", matthews_corrcoef(predicted, true))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
class_=test_images.class_indices.keys()
cm = confusion_matrix(true, predicted)
fig = plt.figure(figsize=(6, 5))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax,cmap="Blues",  fmt='g'); 
ax.set_xlabel('Predicted', fontsize=12)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_, fontsize = 10)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=12)
ax.yaxis.set_ticklabels(class_, fontsize = 10)
plt.yticks(rotation=0)

plt.title('', fontsize=60)

plt.savefig('ConMat24.png')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
from matplotlib import cm
n=len(true)
dim=max(true)+1
result=[]
for i in true:
    result.append([0 for i in range(dim)])
    result[len(result)-1][i]=1
    
    
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes =len(test_images.class_indices.keys())
y_test=np.array(result)
y_pred=np.array(proba)
fpr = dict()
tpr = dict()
roc_auc = dict()

rev={}
for i in class_:
    rev[train_images.class_indices[i]]=i


for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = roc_auc_score(y_test[:, i], y_pred[:, i])


fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred.ravel())
roc_auc["micro"] = roc_auc_score(y_test, y_pred, multi_class='ovr')


cmap = cm.get_cmap('tab10')


plt.figure(figsize=(6,5))
lw = 1
for i in range(n_classes):
    color = cmap(i)
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='{0} (area = {1:0.2f})'
             ''.format(rev[i], roc_auc[i]))

plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
         ''.format(roc_auc["micro"]),
         color=cmap(n_classes), linestyle=':', linewidth=4)

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([-0.1, 1.2])
plt.ylim([-0.1, 1.2])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()